# Introduction
This notebook contains example workflow for web services exposed by the EIDA (European Integrated Data Archive) Federation, focused on the EIDAWS-WFCatalog service and it's functionalities allowing users to filter out low quality and low coverage data before downloading it to the local machine.

In this example we are going to:
1. Retrieve seismic event information from FDSNWS-Event catalogue offered by [GFZ](https://www.gfz-potsdam.de/) EIDA Node.
1. List all stations available in priximity to given event.
1. Using EIDAWS-WFCatalog service, we are going to exclide all station which do not meet following criteria:
    - Sum of gaps = 0
    - Sum of overlaps = 0
    - Availability >= 95%
1. Using FDSNWS-Federator web service, we are going to download:
    - miniSEED file containing the waveforms
    - StationXML file containing station and instrumentation metadata on channel level

# 1. Finding event
Lets start by finding an event. Below two completely different approaches have been presented:
- Downloading event XML document directly from FDSNWS-Event web service
- Downloading event information using ObsPy library


In [8]:
import requests
import xml.etree.ElementTree as ET

ENCODING = "utf-8"
NSMAP = {'mw': 'http://quakeml.org/xmlns/bed/1.2'}

# Define start and end dates
start = "2020-01-01"
end = "2020-06-01"

# Define the bounding box
min_mag = 5
min_lat = 40
max_lat = 45
min_lon = 17
max_lon = 25

events_url = (
    f"http://geofon.gfz-potsdam.de/fdsnws/event/1/query?"
    f"start={start}&end={end}&"
    f"minmag={min_mag}&"
    f"minlat={min_lat}&minlon={min_lon}&"
    f"maxlat={max_lat}&maxlon={max_lon}"
)

# http://geofon.gfz-potsdam.de/fdsnws/event/1/query?start=2020-01-01&end=2020-06-01&minmag=5&minlat=40&minlon=17&maxlat=45&maxlon=25

r = requests.get(events_url, timeout=10)
r.encoding = ENCODING

if r.status_code == 200:
    event_root = ET.fromstring(r.text)
    for event_element in event_root.findall("./mw:eventParameters/*", namespaces=NSMAP):
        # Get event ID from event_element attribute
        event_id = event_element.get("publicID")
        
        # Find event magnitude element
        event_magnitude_element = event_element.find("./mw:magnitude/mw:mag/mw:value", namespaces=NSMAP)
        
        # Get magnitude value from event_magnitude_element
        event_magnitude = event_magnitude_element.text
        
        print(f"Event with ID: {event_id} had magnitude equal to: {event_magnitude}")

Event with ID: smi:org.gfz-potsdam.de/geofon/gfz2020bywr had magnitude equal to: 5.062193141


And now with ObsPy...

In [20]:
from obspy import read_events

evts = read_events(events_url)

for e in evts:
    print(e)

Event:	2020-01-28T20:15:10.670309Z | +41.433,  +19.569 | 5.062193141 M  | automatic

	            resource_id: ResourceIdentifier(id="smi:org.gfz-potsdam.de/geofon/gfz2020bywr")
	          creation_info: CreationInfo(agency_id='GFZ', creation_time=UTCDateTime(2020, 1, 28, 20, 16, 31, 618036))
	    preferred_origin_id: ResourceIdentifier(id="smi:org.gfz-potsdam.de/geofon/Origin/20200128202144.356462.2018953")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:org.gfz-potsdam.de/geofon/Origin/20200128202144.356462.2018953/netMag/M")
	                   ---------
	     event_descriptions: 1 Elements
	                origins: 1 Elements
	             magnitudes: 1 Elements


Users are free to choose how they download data from EIDA, but for convenience we will continue using ObsPy throughoug the rest of this notebook. 🦥